In [65]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import contextily as ctx
import numpy as np

In [66]:
place_name = "Lund, Sweden"
graph = ox.graph_from_place(place_name, network_type="walk")

In [67]:
G = {}

for u, v, data in graph.edges(data=True):
    if u not in G:
        G[u] = [(v, data['length'])]
    else:
        G[u].append((v, data['length']))
    if v not in G:
        G[v] = [(u, data['length'])]
    else:
        G[v].append((u, data['length']))

In [68]:
# Kämnärsvägen 24, Lund
my_x = 55.721008399999995
my_y = 13.2161239

my_node = ox.distance.nearest_nodes(graph, my_x, my_y)

In [69]:
GOAL_DISTANCE = 1000

In [70]:
import math

def haversine_distance(lat1, lon1, lat2, lon2, R=6371000.0):
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi/2.0)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda/2.0)**2
    c = 2.0 * math.atan2(math.sqrt(a), math.sqrt(1.0 - a))
    return R * c  # meter

In [81]:

circumference_nodes = []
tol = 100

for u, data in graph.nodes(data=True):
    x = data['x']
    y = data['y']
    if abs(haversine_distance(my_x, my_y, y, x) - GOAL_DISTANCE / 2) <= tol:
        circumference_nodes.append(u)

In [84]:
import heapq

def dijkstra(start, goal_distance):
    queue = [(0, start, [])] 
    visited = set()

    while queue:
        current_distance, current_node, current_path = heapq.heappop(queue)
        if current_node not in visited:
            visited.add(current_node)
            new_path = current_path + [current_node]

            if current_distance >= goal_distance:
                return current_distance, new_path

            for neighbor, weight in G[current_node]:
                if neighbor not in visited:
                    heapq.heappush(queue, (current_distance + weight, neighbor, new_path))

    raise ValueError("No path found to goal node")

In [ ]:
distance, path = dijkstra(my_node, 1000)